In [ ]:
import pandas as pd
import requests
import glob
from io import BytesIO
import numpy as np
import os
import shutil
import pprint
import json
import warnings

from fastai import *
from fastai.vision import *
warnings.filterwarnings('ignore')

In [ ]:
n_worker = 4
lr = 1e-01
n_cycle = 5
model_name="resnet50"

## Retrieving dataset

In [ ]:
!mkdir -p data
!wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz -P ../data
!tar xzf data/food-101.tar.gz -C data/

## load data

In [ ]:
path = Path('../data/food-101/images')
data = ImageDataBunch.from_folder(path, valid_pct=0.2,
        ds_tfms=get_transforms(), size=224, num_workers=n_worker, bs=64).normalize(imagenet_stats)

## train

In [ ]:
learn = cnn_learner(data, models.resnet50, metrics=error_rate,pretrained=True)

In [ ]:
learn.lr_find()
learn.recorder.plot()


In [ ]:
learn.fit_one_cycle(n_cycle, lr)

In [ ]:
learn.save(f'{model_name}-stage-1')

In [ ]:
learn.load(f'{model_name}-stage-1')

In [ ]:
learn.unfreeze()
learn.lr_find(start_lr=1e-09, end_lr=1e-1)
learn.recorder.plot(skip_end=10)

In [ ]:
learn.fit_one_cycle(n_cycle, max_lr=slice(1e-08,1e-07))

In [ ]:
learn.save(f'{model_name}-stage-2')
learn.load(f'{model_name}-stage-2')

## serializing to pkl model, save list of classes

In [ ]:
final_model_name='model.pkl'
learn.export(final_model_name)

In [ ]:
with open('classes.txt', 'w') as f:
    json.dump(learn.data.classes,f)